In [3]:
from config import (
    azure_endpoint, azure_subscription_key, azure_application, 
    azure_compCode, azure_userID, azure_userNM, azure_serviceType, 
    sender_email
)

import json
import requests 

In [13]:
def evaluate_article(keyword, article_summary):
    """
    키워드와 기사 요약을 평가하여 점수(float)와 해설(str)을 분리하여 반환하는 함수
    
    Args:
        keyword (str): 검색 키워드
        article_summary (str): 기사 요약 텍스트
        
    Returns:
        tuple: (점수(float), 해설(str))
    """
    headers = {
        "Ocp-Apim-Subscription-Key": azure_subscription_key,
        "application": azure_application,
        "compCode": azure_compCode,
        "userID": azure_userID,
        "userNM": azure_userNM,
        "serviceType": azure_serviceType,
        "Content-Type": "application/json; charset=utf-8"
    }
    
    user_content = f"""당신의 역할은 다음과 같습니다:
- "키워드"와 "기사 요약"을 입력값으로 받습니다.
- 기사 요약이 키워드와 얼마나 직접적이고 깊이 있게 연관되는지(키워드 연관성), 그리고 기사 내용이 포장재 회사에 실질적으로 도움이 되는 정보인지(실용성)를 각각 10점 만점으로 평가합니다.
- 두 점수의 평균을 내어 통합점수(0.5점 단위, 소수 첫째자리까지)를 산출합니다.
- 통합점수 기준으로 해당 기사의 '유효성'(유효/참고용/유효하지 않음)을 판단합니다.
- 최종 리턴은 반드시 "X.X점" 형식으로 시작하고, 그 다음 줄부터 해설을 작성하세요.

평가 기준은 다음과 같습니다:
- 키워드 연관성: 기사 요약이 키워드를 직접적으로, 깊이 있게 다루면 10점, 거의 무관하면 1점.
- 실용성: 기사 내용이 포장재 회사 실무에 직접적으로 도움이 되면 10점, 전혀 도움되지 않으면 1점.
- 통합점수 = (키워드 연관성 + 실용성) ÷ 2 (소수 첫째자리)
- 유효성: 통합점수 8점 이상=유효, 5~7.5점=참고용, 5점 미만=유효하지 않음.

---
키워드: {keyword}
기사 요약: {article_summary}
---"""
    
    data = {
        "messages": [
            {"role": "system", "content": "당신은 포장재 산업 전문 분석가입니다. 기사의 연관성과 실용성을 객관적으로 평가합니다."},
            {"role": "user", "content": user_content}
        ],
        "temperature": 0.3
    }
    
    try:
        data_str = json.dumps(data, ensure_ascii=False).encode("utf-8")
        response = requests.post(azure_endpoint, headers=headers, data=data_str)
        response.raise_for_status()
        result = response.json()
        evaluation = result["choices"][0]["message"]["content"].strip()
        
        # 점수와 해설 분리 - 여러 패턴 시도
        import re
        
        # 패턴 1: "X.X점" 형식으로 시작하는 경우
        score_match = re.match(r'^(\d+(\.\d+)?)[점점]\s*(.*)', evaluation, re.DOTALL)
        if score_match:
            score = float(score_match.group(1))
            explanation = score_match.group(3).strip()
            return score, explanation
            
        # 패턴 2: 숫자로 시작하고 줄바꿈이나 대시(—)로 구분된 경우
        score_match = re.match(r'^(\d+(\.\d+)?)\s*[\n\r—\-–]*\s*(.*)', evaluation, re.DOTALL)
        if score_match:
            score = float(score_match.group(1))
            explanation = score_match.group(3).strip()
            return score, explanation
        
        # 그 외의 경우: 첫 줄에서 숫자만 추출
        first_line = evaluation.split('\n', 1)[0].strip()
        score_match = re.search(r'(\d+(\.\d+)?)', first_line)
        if score_match:
            score = float(score_match.group(1))
            # 첫 줄을 제외한 나머지를 설명으로
            if '\n' in evaluation:
                explanation = evaluation.split('\n', 1)[1].strip()
            else:
                # 숫자 부분을 제외한 나머지를 설명으로
                explanation = first_line.replace(score_match.group(0), '').strip()
                if explanation.startswith('점'):
                    explanation = explanation[1:].strip()
            return score, explanation
            
        # 모든 시도 실패 시
        print(f"파싱 실패. 원본 응답: {evaluation}")
        return 0.0, evaluation
        
    except Exception as e:
        print("평가 중 오류 발생:", e)
        return 0.0, f"평가 오류 발생: {str(e)}"

In [15]:
# 테스트 코드
keyword = "Korean rice wine Market"
article_summary = """
Wine Market Forecast - Expected to Grow from $314.34 Billion to $422.08 Billion by 2032, at 4.30% CAGR| E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited
와인 시장 전망 - 2032년까지 3,143억 4천만 달러에서 4,220억 8천만 달러로 성장할 것으로 예상되며, 연평균 성장률(CAGR)은 4.30%입니다. | E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited
업로드 시간: 2025-04-21

원문 URL: openpr.com

요약:

글로벌 와인 시장, 2025년 3,143억 4천만 달러에서 2032년 4,220억 8천만 달러로 성장 예상
연평균 성장률(CAGR) 4.30%
주요 기업: E. & J. Gallo Winery, Concha y Toro, The Wine Group, Bacardi Limited 등
시장 세분화: 제품 유형(스틸 와인, 스파클링 와인, 강화 와인) 및 색상 유형(레드 와인, 화이트 와인, 로제 와인)
지역 분석: 북미, 유럽, 아시아 태평양, 라틴 아메리카, 중동 및 아프리카
보고서의 주요 이점: 시장 동향, 기회, 경쟁 분석, 지역별 수익 매핑 등 제공
"""

score, response = evaluate_article(keyword, article_summary)
print(score)
print(response)

3.0
기사 요약은 글로벌 와인 시장에 대한 전반적인 성장 전망을 다루고 있지만, "Korean rice wine Market"이라는 특정 키워드와는 직접적인 연관성이 부족합니다. 한국의 쌀 와인 시장에 대한 정보는 포함되어 있지 않으며, 전반적인 와인 시장에 대한 내용이 주를 이루고 있습니다. 따라서 키워드 연관성은 2점으로 평가합니다.  
실용성 측면에서도, 포장재 회사에 대한 직접적인 정보나 실무에 도움이 되는 내용이 부족하여 4점으로 평가합니다.  
통합점수는 (2 + 4) ÷ 2 = 3.0점으로, 유효성은 '유효하지 않음'으로 판단됩니다.
